INSTALL
- FAISS - 
- Sent2Vec

In [1]:
import numpy as np
import faiss   

In [2]:
import os
import time
name = "Faiss-EM-TED-bert-vi-en-0.999-"+ str(time.strftime("%Y%m%d-%H%M"))

!mkdir $name
path = os.getcwd() + "/" + name
os.chdir(path)

!pwd


os.chdir(path)


# import os
# os.chdir("/home/lw/clc_fairseq/TED-20210121-1335")

/home/chinh/clc_data/Script/Faiss-EM-TED-bert-vi-en-0.999-20210316-0955


In [3]:
!wget https://github.com/hoangtrungchinh/clc_data/raw/master/en_vi_iwslt.tar.gz
!mkdir data_bin
!tar -xvf  'en_vi_iwslt.tar.gz' -C 'data_bin'

--2021-03-16 09:55:14--  https://github.com/hoangtrungchinh/clc_data/raw/master/en_vi_iwslt.tar.gz
Resolving github.com (github.com)... 13.229.188.59
Connecting to github.com (github.com)|13.229.188.59|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/en_vi_iwslt.tar.gz [following]
--2021-03-16 09:55:14--  https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/en_vi_iwslt.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9044100 (8,6M) [application/octet-stream]
Saving to: ‘en_vi_iwslt.tar.gz’

en_vi_iwslt.tar.gz  100%[===================>]   8,62M  5,86MB/s    in 1,5s    

2021-03-16 09:55:17 (5,86 MB/s) - ‘en_vi_iwslt.tar.gz’ saved [9044100/904410

In [4]:
# import file
import numpy as np
import glob

en_files = sorted(glob.glob("data_bin/*.en"))
vi_files = sorted(glob.glob("data_bin/*.vi"))
print(en_files)
print(vi_files)

lst_en = []
lst_vi = []

start = time.time()

for file in en_files:
  with open(file) as file_in:
    for line in file_in:
      lst_en.append(line)

for file in vi_files:
  with open(file) as file_in:
    for line in file_in:
      lst_vi.append(line)

print(len(lst_en))
print(len(lst_vi))

['data_bin/IWSLT15.TED.tst2012.en-vi.en', 'data_bin/IWSLT15.TED.tst2013.en-vi.en', 'data_bin/IWSLT15.TED.tst2015.en-vi.en', 'data_bin/train.en']
['data_bin/IWSLT15.TED.tst2012.en-vi.vi', 'data_bin/IWSLT15.TED.tst2013.en-vi.vi', 'data_bin/IWSLT15.TED.tst2015.en-vi.vi', 'data_bin/train.vi']
120956
120956


In [5]:
# SPLIT TRAIN, TEST, VALID
from sklearn.model_selection import train_test_split
en_train, en_test_valid, vi_train, vi_test_valid =  train_test_split(lst_en, lst_vi, test_size=0.2, random_state=123)
en_valid, en_test, vi_valid, vi_test = train_test_split(en_test_valid, vi_test_valid, test_size=0.5, random_state=123)
print(len(en_train))
print(len(vi_train))
print(len(en_valid))
print(len(vi_valid))
print(len(en_test))
print(len(vi_test))
print(en_train[20])
print(vi_train[20])
print(en_valid[-1])
print(vi_valid[-1])
print(en_test[-1])
print(vi_test[-1])

96764
96764
12096
12096
12096
12096
And , in fact , when we did some experiments looking at this , what we found is that 46 percent of the variation in how many friends you have is explained by your genes .

Trong thực tế , khi thực hiện một vài thí nghiệm nghiên cứu vấn đề này , chúng tôi nhận ra 46 % sự khác biệt trong số bạn bè của bạn được giải thích bằng gen của bạn .

This is a chart of what it looked like when it first became popular last summer .

Đây là biểu đồ thể hiện điều đó khi video này lần đầu tiên trở nên phổ biến vào mùa hè trước .

And what will happen in the future ?

Chuyện gì sẽ xảy ra trong tương lai ?



In [6]:
file_variable = [en_train,vi_train,en_valid,vi_valid,en_test,vi_test]
file_names = ["en_train","vi_train","en_valid","vi_valid","en_test","vi_test"]

for i in range(len(file_names)):
  with open(file_names[i], 'w') as f:
    for item in file_variable[i]:
      f.write(item)
!ls -al

total 36844
drwxrwxr-x 3 chinh chinh     4096 Thg 3  16 09:55 .
drwxrwxr-x 6 chinh chinh     4096 Thg 3  16 09:55 ..
drwxrwxr-x 2 chinh chinh     4096 Thg 3  16 09:55 data_bin
-rw-rw-r-- 1 chinh chinh  1260181 Thg 3  16 09:55 en_test
-rw-rw-r-- 1 chinh chinh 10011509 Thg 3  16 09:55 en_train
-rw-rw-r-- 1 chinh chinh  1249342 Thg 3  16 09:55 en_valid
-rw-rw-r-- 1 chinh chinh  9044100 Thg 3  16 09:55 en_vi_iwslt.tar.gz
-rw-rw-r-- 1 chinh chinh  1624771 Thg 3  16 09:55 vi_test
-rw-rw-r-- 1 chinh chinh 12896000 Thg 3  16 09:55 vi_train
-rw-rw-r-- 1 chinh chinh  1609835 Thg 3  16 09:55 vi_valid


In [7]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('stsb-bert-large')
# # model = SentenceTransformer('LaBSE')
# #Sentences are encoded by calling model.encode()
# emb1 = model.encode("This is a red cat with a hat.")
# emb2 = model.encode("Have you seen my red cat?")

# cos_sim = util.pytorch_cos_sim(emb1, emb2)
# print("Cosine-Similarity:", cos_sim)

In [8]:
# print(cos_sim.numpy()[0][0])

In [9]:
start = time.time()
srcfile = "vi_train"
tarfile = "en_train"
# lst_srcfile = open(srcfile, "r").readlines()
# lst_tarfile = open(tarfile, "r").readlines()
lst_srcfile = vi_train
lst_tarfile = en_train

vec_lst_srcfile = model.encode(lst_srcfile)
print('=== Ending, Total time (second): ', time.time() - start)

=== Ending, Total time (second):  1380.2327816486359


Test Cosin similarity

In [10]:
import faiss
# print(vec_lst_srcfile.shape[1])
start = time.time()
index = faiss.IndexFlatL2(vec_lst_srcfile.shape[1])
index.ntotal
faiss.normalize_L2(vec_lst_srcfile)
index.add(vec_lst_srcfile)
k = 6
distance, index = index.search(vec_lst_srcfile, k)

print('=== Ending, Total time (second): ', time.time() - start)
print(len(distance))
print(len(index))

print(index)
print(distance)

=== Ending, Total time (second):  50.47859215736389
96764
96764
[[    0 76296 53065 73866 73825 21142]
 [    1 34177 53069 63543  5728  7815]
 [    2 64711 81741 36896 82978 17151]
 ...
 [96761 47887 68505 22300 15498 61197]
 [96762 13228 90924 69061 32898  7876]
 [96763 18922 43565 27245 49732 26097]]
[[0.00000000e+00 8.38295221e-02 8.49540234e-02 8.52618217e-02
  9.17687416e-02 9.20586586e-02]
 [0.00000000e+00 6.74462318e-02 6.80705309e-02 7.02903271e-02
  7.02916384e-02 7.46599436e-02]
 [0.00000000e+00 1.05499506e-01 1.12445354e-01 1.16747022e-01
  1.19321823e-01 1.22380733e-01]
 ...
 [1.19209290e-07 1.34757280e-01 1.57512426e-01 1.64068460e-01
  1.68523550e-01 1.72464132e-01]
 [0.00000000e+00 1.32999659e-01 1.38427019e-01 1.41146064e-01
  1.46415114e-01 1.46845818e-01]
 [0.00000000e+00 8.47262144e-02 8.74986649e-02 8.97815228e-02
  9.39940214e-02 9.42776203e-02]]


In [11]:
i = 6
j = k-1
print(index[i][j])
print("==================================")
print(distance[i][j])
a = vec_lst_srcfile[i]
# print(a)
b = vec_lst_srcfile[index[i][j]]
# TEST
dist_squared = np.sum(np.square(a - b))
print(dist_squared)

24042
0.07960403
0.07960438


In [12]:
start = time.time()
saperate = " || "

src_label = "S"
emb_label = "E"
thres_arr = [0.95, 0.99, 0.995, 0.999]
for i in range(len(thres_arr)):
    threshold = thres_arr[i]
    out_file_write = open("vi_train_EM_"+str(thres_arr[i]), 'w')
    out_file_write_score = open("vi_train_EM_score_"+str(thres_arr[i]), 'w')
    out_file_write_factor = open("vi_train_EM_factor_"+str(thres_arr[i]), 'w')
    num_gth_thres = 0

    for i in range(len(lst_srcfile)):
        score = distance[i][1]
        content = ""
        content_label_arr = []
        if score >= threshold:
            num_gth_thres += 1
            best_simi_index = index[i][1]
            content = lst_srcfile[i].strip() + saperate + lst_tarfile[best_simi_index].strip()

            content_label = [src_label] * len(lst_srcfile[i].split())
            content_label.append(emb_label)
            content_label = content_label + [emb_label] * len(lst_tarfile[best_simi_index].split())

            content_label_arr = " ".join(content_label)
        else:
            content = lst_srcfile[i].strip()
            content_label = [src_label] * len(lst_srcfile[i].split())
            content_label_arr = " ".join(content_label)
 
        out_file_write.writelines(content+ "\n")    
        out_file_write_factor.writelines(content_label_arr + "\n")
        out_file_write_score.writelines(str(score) + "\n")

    print('threshold =', threshold, "EM sentences =", num_gth_thres, "/", len(lst_srcfile), num_gth_thres/len(lst_srcfile), "%" )

    out_file_write.close()
    out_file_write_score.close()
    out_file_write_factor.close()

done = time.time()
elapsed = done - start
print('=== Ending, Total time (second): ', elapsed)

threshold = 0.95 EM sentences = 34 / 96764 0.00035137034434293746 %
threshold = 0.99 EM sentences = 24 / 96764 0.0002480261254185441 %
threshold = 0.995 EM sentences = 23 / 96764 0.00023769170352610475 %
threshold = 0.999 EM sentences = 22 / 96764 0.00022735728163366542 %
=== Ending, Total time (second):  7.181864976882935
